In [46]:
import sys
sys.path.append("../../")

import django
import pylab as pl

from django.db import connections
from firecares.firestation.models import FireDepartment, FireStation
from firecares.utils import dictfetchall
from pandas import DataFrame

%matplotlib inline

django.setup()

print "Number of unmatched fire stations:", FireStation.objects.filter(department__isnull=True).count()

cursor = connections['default'].cursor()

#cursor.execute("enable extension fuzzystrmatch")

queryUnmatchedNearbyStations = """
select a.name as firestation_name, d.geom as firestation_location
from firestation_usgsstructuredata a
inner join firestation_firestation b
  on a.id=b.usgsstructuredata_ptr_id
join firecares_core_address d 
  on b.station_address_id=d.id
where b.department_id is null and d.geom is not null and 
ST_DISTANCE(ST_TRANSFORM(d.geom,900913),ST_TRANSFORM(ST_GEOMFROMTEXT(%s,900913),900913)) * 0.000621371 <= 15 and
levenshtein(a.name,%s) <= 20;
"""
#
queryDepartmentsWithGeo = """
select c.name as department_name, c.id as dep_id, ST_ASTEXT(ST_TRANSFORM(e.geom,900913)) as headquarters_location
from firestation_firedepartment c 
join firecares_core_address e 
  on c.headquarters_address_id=e.id
where e.geom is not null; 
"""


cursor.execute(queryDepartmentsWithGeo)
fireDepartments = dictfetchall(cursor)
df = DataFrame(fireDepartments)
print df.size

# distribution of levenshtein distances
#pl.xlabel("Levenshtein Distance")
#pl.ylabel("Count of records")
totalMatchedStations = 0;
for fireDepartment in fireDepartments:
    print fireDepartment['department_name']
    params = [fireDepartment['headquarters_location'],fireDepartment['department_name']]
    reults = cursor.execute(queryUnmatchedNearbyStations,params)
    nearbyStations = dictfetchall(cursor)
    df = DataFrame(nearbyStations)
    print "Matched Stations:", df.size
    totalMatchedStations += df.size

print "Total Matched Stations:", totalMatchedStations
#for index, row in df.iterrows():
 #   print row['firestation_name'] , row['firestation_address']





Number of unmatched fire stations: 50743
81492
Oklahoma City Fire Department
Matched Stations: 32
McGregor Volunteer Fire Department
Matched Stations: 2
Oak Grove Fire District
Matched Stations: 4
Grantfork Volunteer Fire Department
Matched Stations: 6
Greater Peoria Airport Fire Department
Matched Stations: 10
Ada Fire Department
Matched Stations: 22
Hamel Community Fire Protection District
Matched Stations: 10
Thelma Volunteer Fire Department
Matched Stations: 28
Thousandsticks Volunteer Fire & Rescue Department
Matched Stations: 4
Adrian Fire Department
Matched Stations: 22
Arab Fire Department
Matched Stations: 8
Corning Fire Department
Matched Stations: 30
Anaheim Fire & Rescue
Matched Stations: 22
Apple Valley Fire Protection District
Matched Stations: 14
Utah County Fire Department
Matched Stations: 14
Linton Township Volunteer Fire Protection Company  Inc.
Matched Stations: 2
Fountain Run Volunteer Fire Department
Matched Stations: 20
Jenkins Fire Department
Matched Stations: 1

KeyboardInterrupt: 